In [ ]:
# Clase que representa un nodo en el árbol AVL, donde cada nodo contiene información de una película.
class Movie:
    def __init__(self, title, year, worldwide_earnings, domestic_earnings, foreign_earnings, domestic_percent_earnings, foreign_percent_earnings, left=None, right=None, height=1):
        # Atributos de la película
        self.title = title
        self.year = year
        self.worldwide_earnings = worldwide_earnings
        self.domestic_earnings = domestic_earnings
        self.foreign_earnings = foreign_earnings
        self.domestic_percent_earnings = domestic_percent_earnings
        self.foreign_percent_earnings = foreign_percent_earnings
        # Punteros a los hijos izquierdo y derecho en el árbol
        self.left = left
        self.right = right
        # Altura del nodo, utilizada para mantener balanceado el árbol AVL
        self.height = height

# Clase que gestiona el árbol AVL y las operaciones de inserción, eliminación y búsqueda
class MovieActions:
    def __init__(self):
        # Raíz del árbol AVL, inicialmente es None porque el árbol está vacío
        self.root = None

    # Método para crear un nuevo nodo Movie
    def create_node(self, title, year, worldwide_earnings, domestic_earnings, foreign_earnings, domestic_percent_earnings, foreign_percent_earnings):
        return Movie(title, year, worldwide_earnings, domestic_earnings, foreign_earnings, domestic_percent_earnings, foreign_percent_earnings)

    # Método para obtener la altura de un nodo, si el nodo es None, retorna 0
    def get_height(self, node):
        if not node:
            return 0
        return node.height

    # Retorna la altura máxima entre dos nodos
    def max_height(self, a, b):
        return max(a, b)

    # Calcula el balance de un nodo, que es la diferencia de alturas entre los subárboles izquierdo y derecho
    def get_balance(self, node):
        if not node:
            return 0
        return self.get_height(node.left) - self.get_height(node.right)

    # Rotación a la derecha para balancear el árbol AVL
    def right_rotate(self, y):
        x = y.left  # Guardamos el hijo izquierdo de y
        T2 = x.right  # Guardamos el subárbol derecho de x

        # Realizamos la rotación
        x.right = y
        y.left = T2

        # Actualizamos las alturas de los nodos
        y.height = self.max_height(self.get_height(y.left), self.get_height(y.right)) + 1
        x.height = self.max_height(self.get_height(x.left), self.get_height(x.right)) + 1

        # Retornamos el nuevo nodo raíz después de la rotación
        return x

    # Rotación a la izquierda para balancear el árbol AVL
    def left_rotate(self, x):
        y = x.right  # Guardamos el hijo derecho de x
        T2 = y.left  # Guardamos el subárbol izquierdo de y

        # Realizamos la rotación
        y.left = x
        x.right = T2

        # Actualizamos las alturas de los nodos
        x.height = self.max_height(self.get_height(x.left), self.get_height(x.right)) + 1
        y.height = self.max_height(self.get_height(y.left), self.get_height(y.right)) + 1

        # Retornamos el nuevo nodo raíz después de la rotación
        return y

    # Método para buscar un nodo por su título en el árbol AVL
    def search(self, title):
        p, pad = self.root, None
        # Recorremos el árbol buscando el nodo con el título especificado
        while p is not None:
            if title == p.title:
                return p, pad  # Retorna el nodo encontrado y su padre
            else:
                pad = p  # Actualizamos el nodo padre
                if title < p.title:
                    p = p.left  # Nos movemos al subárbol izquierdo
                else:
                    p = p.right  # Nos movemos al subárbol derecho
        return p, pad  # Si no se encuentra, retorna None y el último padre

    # Método para insertar una nueva película (nodo) en el árbol AVL
    def insert_movie(self, movie):
        node, parent = self.search(movie.title)
        # Si el nodo ya existe, no lo insertamos (evita duplicados)
        if node is not None:
            return

        # Si el árbol está vacío, el nuevo nodo se convierte en la raíz
        if parent is None:
            self.root = movie
        else:
            # Si no está vacío, colocamos el nodo en la posición correcta
            if movie.title < parent.title:
                parent.left = movie
            else:
                parent.right = movie

        # Balanceamos el árbol después de la inserción
        self.root = self._balance_after_insert(self.root, movie.title)

    # Método auxiliar para balancear el árbol después de la inserción
    def _balance_after_insert(self, node, title):
        if not node:
            return node

        # Actualizamos la altura del nodo
        node.height = 1 + self.max_height(self.get_height(node.left), self.get_height(node.right))

        # Obtenemos el balance del nodo para verificar si está desbalanceado
        balance = self.get_balance(node)

        # Realizamos las rotaciones necesarias si el nodo está desbalanceado
        if balance > 1 and title < node.left.title:
            return self.right_rotate(node)

        if balance < -1 and title > node.right.title:
            return self.left_rotate(node)

        if balance > 1 and title > node.left.title:
            node.left = self.left_rotate(node.left)
            return self.right_rotate(node)

        if balance < -1 and title < node.right.title:
            node.right = self.right_rotate(node.right)
            return self.left_rotate(node)

        return node

    # Método para eliminar un nodo (película) del árbol AVL por título
    def delete_movie(self, title):
        node, parent = self.search(title)
        # Si el nodo no existe, no se realiza ninguna acción
        if node is None:
            return

        # Llamamos al método auxiliar que realiza la eliminación y rebalanceo
        self.root = self._delete_movie(self.root, title)

    # Método auxiliar para eliminar un nodo de manera recursiva
    def _delete_movie(self, root, title):
        if not root:
            return root

        # Recursivamente buscamos el nodo a eliminar
        if title < root.title:
            root.left = self._delete_movie(root.left, title)
        elif title > root.title:
            root.right = self._delete_movie(root.right, title)
        else:
            # Caso 1: Nodo con un solo hijo o sin hijos
            if not root.left:
                return root.right
            elif not root.right:
                return root.left

            # Caso 2: Nodo con dos hijos, obtenemos el predecesor
            temp = self.get_max_value_node(root.left)
            # Reemplazamos el valor del nodo por el predecesor
            root.title = temp.title
            root.year = temp.year
            root.worldwide_earnings = temp.worldwide_earnings
            root.domestic_earnings = temp.domestic_earnings
            root.foreign_earnings = temp.foreign_earnings
            root.domestic_percent_earnings = temp.domestic_percent_earnings
            root.foreign_percent_earnings = temp.foreign_percent_earnings

            # Eliminamos el predecesor
            root.left = self._delete_movie(root.left, temp.title)

        # Actualizamos la altura del nodo
        root.height = 1 + self.max_height(self.get_height(root.left), self.get_height(root.right))

        # Rebalanceamos el árbol si es necesario
        balance = self.get_balance(root)

        if balance > 1 and self.get_balance(root.left) >= 0:
            return self.right_rotate(root)

        if balance > 1 and self.get_balance(root.left) < 0:
            root.left = self.left_rotate(root.left)
            return self.right_rotate(root)

        if balance < -1 and self.get_balance(root.right) <= 0:
            return self.left_rotate(root)

        if balance < -1 and self.get_balance(root.right) > 0:
            root.right = self.right_rotate(root.right)
            return self.left_rotate(root)

        return root

    # Método auxiliar para encontrar el nodo con el valor máximo en un subárbol (predecesor)
    def get_max_value_node(self, node):
        current = node
        while current.right is not None:
            current = current.right
        return current

    # Método para rellenar el árbol AVL con las primeras 20 líneas de un archivo CSV
    def fill_nodes(self, csv_file):
        try:
            with open(csv_file, 'r') as file:
                count = 0
                # Leemos el archivo línea por línea
                for line in file:
                    if count >= 20:
                        break
                    # Dividimos la línea por comas para obtener los atributos de la película
                    parts = line.strip().split(',')

                    # Extraemos los atributos de la película desde el archivo CSV
                    title = parts[0]
                    year = int(parts[1])
                    worldwide_earnings = float(parts[2])
                    domestic_earnings = float(parts[3])
                    foreign_earnings = float(parts[4])
                    domestic_percent_earnings = float(parts[5])
                    foreign_percent_earnings = float(parts[6])

                    # Creamos un nuevo nodo Movie y lo insertamos en el árbol
                    movie = self.create_node(title, year, worldwide_earnings, domestic_earnings, foreign_earnings, domestic_percent_earnings, foreign_percent_earnings)
                    self.insert_movie(movie)

                    count += 1
        except Exception as e:
            print(f"Error reading file: {e}")

